In [35]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import bs4 as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [36]:
#get an url list 
def get_url_list(page1_url):
    var_url = '&section_offset=5&items_offset='
    url_lst = [page1_url]
    for page in range(1,17):
        url = page1_url+var_url+str(18*page)
        url_lst.append(url)
    return url_lst

In [37]:

def get_house_info(url, price):
    #url = 'https://www.airbnb.com/rooms/3503946?s=P8Cyps_7'
    import requests
    from bs4 import BeautifulSoup
    import re
    response = requests.get(url)
    output = []
    if not response.status_code == 200:
        print('failed to get the fidelity website')
        return output
    else:
        results_page = BeautifulSoup(response.content,'html5lib')
        # rating
        if len(results_page.find_all('div', class_="_l0ao8q")) == 0:
            rating = 'no rating'
        else:
            rating_sec = results_page.find_all('div', class_="_l0ao8q")[0]
            rating = rating_sec.find('span').get('aria-label')
        # coordinates
        coordinates = re.search(
            '"listing_lat":([0-9]{2}.[0-9]*),"listing_lng":(-[0-9]{2}.[0-9]*),', 
            response.text)
        
        lat = coordinates.group(1)
        lon = coordinates.group(2)
        # host
        superhost = re.search('"is_superhost":(.{1,5})', 
                              response.text).group(1)
        
        if len(results_page.find_all('span', class_="_fgdupie"))==0:
            response_rate = "no info"
        else:
            host = results_page.find_all('span', class_="_fgdupie")
            #response rate
            response_rate = float('nan')
            for host_ in host[::-1]:
                if 'Response rate' in host_.text:
                    response_rate = host_.text.split(':')[-1]
                    break
                
      
        if re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text) != None:
            accommodates = re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text).group(1)
        else:
            accommodates = re.search('"guest_label":"([0-9][0-9]?) guest.*', response.text).group(1)
        #print (accommodates)
        
        
        #bedrooms
        if re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text) != None:
            bedrooms = (re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text)).group(1)
        else:
            bedrooms = "studio"
            #print (bedrooms)
            
        #bathrooms
        try:
            bathrooms = (re.search('"bathroom_label":"([0-9]\.?[0-9]?).*","bed_label"', response.text)).group(1)
        except:
            bathrooms = re.search('"bathroom_label":"([0-9]\.?[0-9]?).*","bed_label"', response.text)
        #beds
        
        try:
            beds = (re.search('"bed_label":"(.).*","bedroom_label"', response.text)).group(1)
        except:
            beds = re.search('"bed_label":"(.).*","bedroom_label"', response.text)
        #print (beds)
        
        #room type
        try:
            room_type = results_page.find('span',class_="_1hh2h7tb").text
        except AttributeError:
            room_type='NoneType'

        #number of review
        try:
            number_of_reviews = results_page.find('div', class_="_vy3ibx").text
        except AttributeError:
            number_of_reviews='NoneType'
        
        output=[price, accommodates, bedrooms,beds, bathrooms, room_type, 
                number_of_reviews,  rating, lat, lon, superhost, response_rate]
        #print('output=')
        #print(output)
        
    return output

In [38]:
def get_prices_urls(url,dataframe):
    info_list=[]
    from selenium import webdriver  
    browser = webdriver.Chrome('/Users/daqiliu/Desktop/chromedriver')
    #url='https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&allow_override%5B%5D=&price_min=100&price_max=199&s_tag=qfon6p2A'
    browser.get(url)
    html_source = browser.page_source
    #browser.quit()
    #get the prices for every apartment since the price infomation is not availabe inside the apt page
    prices = browser.find_elements(By.XPATH,"//span[@class='_17oldnte']")
    price_list = []
    for price in prices:
        try:
            price = price.text
            price_list.append(price)
        except:
            pass
    price_list = price_list[::3]
    browser.quit()
    print("length of price list is: ", len(price_list))

    #get the url for every apt on this page
    results_page = BeautifulSoup(html_source, "lxml")
    
    #print (results_page)
    
    urls = results_page.find_all("meta", content=re.compile("www.airbnb.com/rooms"))
    #prices = (re.findall('"amount_formatted":"\$([0-9]{2,6})",', results_page.text))
    #print(prices)
    #urls = browser.find_elements(By.XPATH,"//a[@class='_1ol0z3h']")
    url_list = []
    for url in urls:
        url = str(url)
        url = "https://"+url[15:-18]
        #print (x)
        url_list.append(url)
    print("length of apt_url_list is:" + str(len(url_list)))
    print("-----------------")
    #print(url_list)
    #pass url and price to the get_house_info function and extract the rest infomation 
    
    if (len(price_list) == len(url_list)) and (len(price_list)>0):
        for x in range(len(price_list)):
            #print(get_house_info(url_list[x], price_list[x]))
            info_list.append(get_house_info(url_list[x], price_list[x]))
        #print('info list is:')
        #print(info_list)
        df2=pd.DataFrame(info_list,columns=['price','accommodates', 'bedrooms','beds', 
                                            'bathrooms', 'room_type', 'number_of_reviews',  'rating', 
                                            'lat', 'lon', 'superhost', 'response_rate'])
        #print('dataframe is:')
        #print(df2)
        
        dataframe=pd.concat([dataframe,df2], ignore_index=True)
        print('complete dataframe')

    #print("info_list=")
    #print(info_list)
    return dataframe  

In [51]:
page_url="https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&adults=0&children=0&infants=0&guests=0&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&price_min=917&allow_override%5B%5D=&s_tag=eSRH0a1_"
url_list=get_url_list(page_url)
df = pd.DataFrame(columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 
                           'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 
                           'superhost', 'response_rate'])
for url in url_list:
    df=get_prices_urls(url,df)

length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  3
length of apt_url_list is:3
-----------------
complete dataframe
length of price list is:  0
length of apt_url_list i

In [52]:
df

,price,accommodates,bedrooms,beds,bathrooms,room_type,number_of_reviews,rating,lat,lon,superhost,response_rate
0,Price\n$995 per night,0,4,4,5,Entire townhouse,44 Reviews,Rating 5 out of 5,40.73220168085883,-74.00237485584977,false,100%
1,"Price\n$1,099 per night",8,3,3,2,Entire loft,142 Reviews,Rating 4.5 out of 5,40.72328169898259,-73.99851702120611,false,100%
2,"Price\n$1,000 per night",4,2,2,1,Entire loft,94 Reviews,Rating 4.5 out of 5,40.74968308935695,-73.99380793934897,false,94%
3,Price\n$950 per night,8,4,6,3.5,Entire townhouse,36 Reviews,Rating 5 out of 5,40.72592084252041,-73.9799117182836,"true,",100%
4,"Price\n$1,295 per night",5,2,2,3,Entire loft,19 Reviews,Rating 5 out of 5,40.72560405043226,-73.99486772073533,"true,",100%
5,Price\n$995 per night,4,2,2,2.5,Entire condominium,16 Reviews,Rating 5 out of 5,40.76681632749283,-73.97942598478666,"true,",100%
6,"Price\n$2,250 per night",8,4,4,3.5,Entire loft,10 Reviews,Rating 5 out of 5,40.72206783086665,-74.00232007729227,false,100%
7,"Price\n$1,000 per night",6,5,6,3,Entire loft,76 Reviews,Rating 4.5 out of 5,40.73443868087639,-74.00128928440927,false,60%
8,Price\n$995 per night,3,1,2,2,Entire loft,6 Reviews,Rating 5 out of 5,40.72390035209946,-73.99677988216276,false,100%
9,Price\n$995 per night,8,4,4,3,Entire serviced apartment,88 Reviews,Rating 5 out of 5,40.756597499623226,-73.98329038538897,false,100%


In [53]:
df.to_csv('/Users/daqiliu/Desktop/airbnbdata_above933.csv')